# 08 - Predictive Modeling (Phase 3 Implementation)

This notebook now implements the **Phase 3** predictive modeling for time-series forecasting and customer behavior prediction, building on the methodology planning and data preparation completed in Phase 2.

## Objectives
- Document predictive modeling approach
- Implement time series forecasting models using **ARIMA** and **Prophet**
- Implement **customer behavior prediction** (churn risk / activity) model
- Define and compute evaluation metrics
- Use prepared datasets for end-to-end modeling and evaluation

## Phase Progress
- ✅ Phase 2: Model planning and approach documentation
- ✅ Phase 2: Methodology selection and justification
- ✅ Phase 2: Evaluation metrics definition
- ✅ Phase 2: Data preparation for modeling
- ✅ Phase 3: ARIMA-based daily revenue forecasting implementation
- ✅ Phase 3: Prophet-based daily revenue forecasting implementation
- ✅ Phase 3: Customer behavior prediction baseline model


In [5]:
# Load required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from scipy.stats import randint, uniform

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("=" * 80)
print("PREDICTIVE MODELING - PHASE 3 IMPLEMENTATION")
print("=" * 80)

# Load data
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
data_path = os.path.join(project_root, 'data', 'raw', 'Online Retail.csv')

df = pd.read_csv(data_path, encoding='latin-1')
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]
df = df[df['Description'].notna()]
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
df = df[df['InvoiceDate'].notna()]

print(f"\nDataset loaded: {df.shape[0]:,} transactions")
print(f"Date range: {df['InvoiceDate'].min()} to {df['InvoiceDate'].max()}")
print("Target series: DailyRevenue (aggregated from TotalPrice)")
print("Customer features prepared for churn / activity prediction")
print(f"\nNote: This notebook implements Phase 3 models: ARIMA, Prophet, and a customer behavior classifier.")


ModuleNotFoundError: No module named 'statsmodels'

## Step 1: Predictive Modeling Objectives

Define the key predictive modeling goals for Phase 3.


In [ ]:
# Modeling objectives
print("=" * 80)
print("PREDICTIVE MODELING OBJECTIVES")
print("=" * 80)

objectives = {
    "1. Demand Forecasting": {
        "Goal": "Predict future daily/monthly revenue and transaction volumes",
        "Use Case": "Stock planning, inventory optimization",
        "Time Horizon": "Short-term (1-30 days), Medium-term (1-6 months)"
    },
    "2. Customer Behavior Prediction": {
        "Goal": "Predict customer purchase likelihood and churn risk",
        "Use Case": "Targeted marketing, retention campaigns",
        "Time Horizon": "Next purchase timing, 30/60/90 day churn"
    },
    "3. Product Demand Prediction": {
        "Goal": "Forecast product-level demand",
        "Use Case": "Product-specific stock allocation",
        "Time Horizon": "Weekly and monthly forecasts"
    },
    "4. Basket Size Prediction": {
        "Goal": "Predict transaction value and quantity",
        "Use Case": "Revenue forecasting, pricing strategies",
        "Time Horizon": "Next transaction prediction"
    }
}

for obj_name, details in objectives.items():
    print(f"\n{obj_name}:")
    for key, value in details.items():
        print(f"  {key}: {value}")

print("\n" + "=" * 80)


### Model Choice Given Data Limitations

The dataset covers **only a single year of transactions**, which limits the reliability of long-horizon time-series models like ARIMA/Prophet for pure forecasting. We therefore:
- Use ARIMA/Prophet **exploratorily** to understand short-term patterns.
- Rely primarily on **classical machine learning models (Random Forest, XGBoost)** for **customer behavior prediction**, where we can engineer features from the available year (recency, frequency, monetary, etc.).
- Avoid deep learning models because, with this data size and horizon, they are more likely to **overfit** and provide less interpretable results for business stakeholders.

This design keeps the modeling aligned with the data volume while still giving actionable, interpretable insights for retail decision-making.


## Step 2: Model Selection & Justification

Document planned models and justify their selection.


In [ ]:
# Model selection
print("=" * 80)
print("MODEL SELECTION & JUSTIFICATION")
print("=" * 80)

models = {
    "Time Series Forecasting": {
        "ARIMA": {
            "Description": "AutoRegressive Integrated Moving Average",
            "Justification": "Handles trend and seasonality, interpretable, good for univariate time series",
            "Use Case": "Daily/monthly revenue forecasting",
            "Limitations": "Requires stationarity, assumes linear relationships"
        },
        "Prophet": {
            "Description": "Facebook's time series forecasting tool",
            "Justification": "Handles seasonality, holidays, trend changes automatically, robust to missing data",
            "Use Case": "Revenue forecasting with multiple seasonality patterns",
            "Limitations": "Less interpretable than ARIMA, requires sufficient historical data"
        }
    },
    "Customer Behavior": {
        "Logistic Regression": {
            "Description": "Binary classification for churn prediction",
            "Justification": "Interpretable, handles categorical features well, baseline model",
            "Use Case": "Customer churn prediction",
            "Limitations": "Assumes linear relationships, may need feature engineering"
        },
        "Random Forest": {
            "Description": "Ensemble method for classification/regression",
            "Justification": "Handles non-linear relationships, feature importance, robust to outliers",
            "Use Case": "Purchase likelihood, basket size prediction",
            "Limitations": "Less interpretable, can overfit with small datasets"
        }
    }
}

for category, model_dict in models.items():
    print(f"\n{category}:")
    print("=" * 60)
    for model_name, details in model_dict.items():
        print(f"\n{model_name}:")
        for key, value in details.items():
            print(f"  {key}: {value}")

print("\n" + "=" * 80)


MODEL SELECTION & JUSTIFICATION

Time Series Forecasting:

ARIMA:
  Description: AutoRegressive Integrated Moving Average
  Justification: Handles trend and seasonality, interpretable, good for univariate time series
  Use Case: Daily/monthly revenue forecasting
  Limitations: Requires stationarity, assumes linear relationships

Prophet:
  Description: Facebook's time series forecasting tool
  Justification: Handles seasonality, holidays, trend changes automatically, robust to missing data
  Use Case: Revenue forecasting with multiple seasonality patterns
  Limitations: Less interpretable than ARIMA, requires sufficient historical data

Customer Behavior:

Logistic Regression:
  Description: Binary classification for churn prediction
  Justification: Interpretable, handles categorical features well, baseline model
  Use Case: Customer churn prediction
  Limitations: Assumes linear relationships, may need feature engineering

Random Forest:
  Description: Ensemble method for classificat

## Step 3: Data Preparation for Modeling

Prepare time-series and customer-level datasets for modeling.


In [ ]:
# Data preparation
print("=" * 80)
print("DATA PREPARATION FOR MODELING")
print("=" * 80)

# 1. Time-series data for forecasting
print("\n1. TIME-SERIES DATA PREPARATION:")
daily_data = df.groupby(df['InvoiceDate'].dt.date).agg({
    'TotalPrice': 'sum',
    'Quantity': 'sum',
    'InvoiceNo': 'nunique',
    'CustomerID': 'nunique'
}).reset_index()

daily_data.columns = ['Date', 'DailyRevenue', 'DailyQuantity', 'DailyTransactions', 'DailyCustomers']
daily_data['Date'] = pd.to_datetime(daily_data['Date'])
daily_data = daily_data.sort_values('Date').reset_index(drop=True)

# Create complete date range
date_range = pd.date_range(start=daily_data['Date'].min(), end=daily_data['Date'].max(), freq='D')
daily_complete = pd.DataFrame({'Date': date_range})
daily_complete = daily_complete.merge(daily_data, on='Date', how='left')
daily_complete = daily_complete.fillna(0)

print(f"  Daily time-series: {len(daily_complete)} days")
print(f"  Date range: {daily_complete['Date'].min()} to {daily_complete['Date'].max()}")
print(f"  Missing days filled: {len(daily_complete) - len(daily_data)}")

# 2. Monthly aggregation
monthly_data = df.groupby(df['InvoiceDate'].dt.to_period('M')).agg({
    'TotalPrice': 'sum',
    'Quantity': 'sum',
    'InvoiceNo': 'nunique',
    'CustomerID': 'nunique'
}).reset_index()

monthly_data.columns = ['YearMonth', 'MonthlyRevenue', 'MonthlyQuantity', 'MonthlyTransactions', 'MonthlyCustomers']
monthly_data['Date'] = pd.to_datetime(monthly_data['YearMonth'].astype(str))
print(f"\n  Monthly time-series: {len(monthly_data)} months")

# 3. Customer-level features for behavior prediction
print("\n2. CUSTOMER-LEVEL FEATURES:")
reference_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

# Use named aggregations to avoid duplicate column keys and ensure clean feature names
customer_features = df.groupby('CustomerID').agg(
    Recency=('InvoiceDate', lambda x: (reference_date - x.max()).days),  # days since last purchase
    Frequency=('InvoiceNo', 'nunique'),                                  # number of distinct invoices
    TotalSpent=('TotalPrice', 'sum'),                                    # total monetary value
    AvgTransaction=('TotalPrice', 'mean'),                               # average basket value
    TotalQuantity=('Quantity', 'sum'),                                   # total units purchased
    FirstPurchase=('InvoiceDate', 'min'),                                # first purchase date
    LastPurchase=('InvoiceDate', 'max')                                  # most recent purchase date
).reset_index()

customer_features['CustomerLifetime'] = (customer_features['LastPurchase'] - customer_features['FirstPurchase']).dt.days
customer_features['AvgDaysBetweenPurchases'] = customer_features['CustomerLifetime'] / customer_features['Frequency']

print(f"  Customer features: {len(customer_features)} customers")
print(f"  Features: Recency, Frequency, Monetary, Lifetime, AvgDaysBetweenPurchases")

# Display sample
print("\nSample Time-Series Data (Last 10 days):")
print(daily_complete[['Date', 'DailyRevenue', 'DailyTransactions']].tail(10).to_string(index=False))

print("\nSample Customer Features (Top 10 by Total Spent):")
print(customer_features.nlargest(10, 'TotalSpent')[['CustomerID', 'Recency', 'Frequency', 'TotalSpent']].to_string(index=False))

print("\n" + "=" * 80)
print("DATA PREPARATION COMPLETE")
print("=" * 80)


DATA PREPARATION FOR MODELING

1. TIME-SERIES DATA PREPARATION:


NameError: name 'df' is not defined

In [ ]:
# Evaluation metrics
print("=" * 80)
print("EVALUATION METRICS")
print("=" * 80)

metrics = {
    "Time Series Forecasting": {
        "MAE (Mean Absolute Error)": "Average absolute difference between predicted and actual values",
        "RMSE (Root Mean Squared Error)": "Penalizes larger errors more, good for business impact",
        "MAPE (Mean Absolute Percentage Error)": "Percentage error, interpretable for stakeholders",
        "R² (Coefficient of Determination)": "Proportion of variance explained by the model"
    },
    "Classification (Churn/Purchase Prediction)": {
        "Accuracy": "Overall correctness of predictions",
        "Precision": "Proportion of positive predictions that are correct",
        "Recall": "Proportion of actual positives correctly identified",
        "F1-Score": "Harmonic mean of precision and recall",
        "ROC-AUC": "Area under ROC curve, measures classification performance"
    },
    "Regression (Basket Size)": {
        "MAE": "Average absolute error in basket size prediction",
        "RMSE": "Penalizes larger errors",
        "R²": "Model fit quality"
    }
}

for category, metric_dict in metrics.items():
    print(f"\n{category}:")
    print("-" * 60)
    for metric, description in metric_dict.items():
        print(f"  • {metric}: {description}")

print("\n" + "=" * 80)


## Step 5: Modeling Approach & Implementation Plan

Document the step-by-step approach for Phase 3 implementation.


In [ ]:
# Implementation plan
print("=" * 80)
print("MODELING APPROACH & IMPLEMENTATION PLAN")
print("=" * 80)

plan = {
    "Phase 1: Baseline Models": [
        "1. Implement ARIMA for daily revenue forecasting",
        "2. Train baseline logistic regression for churn prediction",
        "3. Evaluate baseline models using defined metrics",
        "4. Document baseline performance"
    ],
    "Phase 2: Advanced Models": [
        "1. Implement Prophet for revenue forecasting with seasonality",
        "2. Train Random Forest for purchase likelihood prediction",
        "3. Compare advanced models with baselines",
        "4. Feature engineering and hyperparameter tuning"
    ],
    "Phase 3: Model Validation": [
        "1. Time-series cross-validation (walk-forward validation)",
        "2. Hold-out test set evaluation",
        "3. Statistical significance testing",
        "4. Business impact assessment"
    ],
    "Phase 4: Model Deployment": [
        "1. Model serialization and versioning",
        "2. Prediction pipeline development",
        "3. Model monitoring framework",
        "4. Documentation and reporting"
    ]
}

for phase, steps in plan.items():
    print(f"\n{phase}:")
    print("-" * 60)
    for step in steps:
        print(f"  {step}")

print("\n" + "=" * 80)
print("KEY CONSIDERATIONS:")
print("=" * 80)
considerations = [
    "Train/Test Split: Use temporal split (e.g., last 3 months as test set)",
    "Cross-Validation: Time-series cross-validation to avoid data leakage",
    "Feature Engineering: Create lag features, rolling statistics, temporal features",
    "Model Interpretability: Balance accuracy with interpretability for business stakeholders",
    "Scalability: Ensure models can handle production-scale data",
    "Monitoring: Plan for model performance monitoring and retraining"
]

for i, consideration in enumerate(considerations, 1):
    print(f"{i}. {consideration}")

print("\n" + "=" * 80)
print("PREDICTIVE MODELING PLANNING COMPLETE")
print("=" * 80)
print("\nNext Steps (Phase 3):")
print("  1. Implement baseline ARIMA model")
print("  2. Implement Prophet model")
print("  3. Build customer churn prediction model")
print("  4. Evaluate and compare all models")
print("  5. Document results and business implications")


## Step 6: Data Visualization for Modeling Preparation

Visualize prepared datasets to understand their characteristics for modeling.


In [ ]:
# Visualize prepared datasets
print("=" * 80)
print("DATA VISUALIZATION FOR MODELING PREPARATION")
print("=" * 80)

# 1. Time-series data visualization
fig, axes = plt.subplots(3, 2, figsize=(16, 18))
fig.suptitle('Predictive Modeling - Data Preparation Visualization', fontsize=16, y=0.995)

# Daily revenue time-series
axes[0, 0].plot(daily_complete['Date'], daily_complete['DailyRevenue'], 
               linewidth=1.5, color='steelblue', alpha=0.7)
axes[0, 0].set_xlabel('Date', fontsize=12)
axes[0, 0].set_ylabel('Daily Revenue (£)', fontsize=12)
axes[0, 0].set_title('Daily Revenue Time-Series', fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# Monthly revenue time-series
axes[0, 1].plot(monthly_data['Date'], monthly_data['MonthlyRevenue'], 
               marker='o', linewidth=2, markersize=6, color='coral')
axes[0, 1].set_xlabel('Date', fontsize=12)
axes[0, 1].set_ylabel('Monthly Revenue (£)', fontsize=12)
axes[0, 1].set_title('Monthly Revenue Time-Series', fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Daily revenue distribution
axes[1, 0].hist(daily_complete['DailyRevenue'], bins=50, edgecolor='black', alpha=0.7, color='teal')
axes[1, 0].set_xlabel('Daily Revenue (£)', fontsize=12)
axes[1, 0].set_ylabel('Frequency', fontsize=12)
axes[1, 0].set_title('Daily Revenue Distribution', fontweight='bold')
axes[1, 0].axvline(x=daily_complete['DailyRevenue'].mean(), color='red', linestyle='--', 
                  linewidth=2, label=f'Mean: £{daily_complete["DailyRevenue"].mean():,.0f}')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Monthly revenue distribution
axes[1, 1].hist(monthly_data['MonthlyRevenue'], bins=20, edgecolor='black', alpha=0.7, color='purple')
axes[1, 1].set_xlabel('Monthly Revenue (£)', fontsize=12)
axes[1, 1].set_ylabel('Frequency', fontsize=12)
axes[1, 1].set_title('Monthly Revenue Distribution', fontweight='bold')
axes[1, 1].axvline(x=monthly_data['MonthlyRevenue'].mean(), color='red', linestyle='--',
                  linewidth=2, label=f'Mean: £{monthly_data["MonthlyRevenue"].mean():,.0f}')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3, axis='y')

# Customer features distribution
axes[2, 0].scatter(customer_features['Recency'], customer_features['Frequency'],
                  c=customer_features['TotalSpent'], cmap='viridis', alpha=0.6, s=20, edgecolors='black', linewidth=0.3)
axes[2, 0].set_xlabel('Recency (days)', fontsize=12)
axes[2, 0].set_ylabel('Frequency (transactions)', fontsize=12)
axes[2, 0].set_title('Customer Features: Recency vs Frequency\n(Color = Total Spent)', fontweight='bold')
axes[2, 0].set_xscale('log')
axes[2, 0].set_yscale('log')
axes[2, 0].grid(True, alpha=0.3)
cbar = plt.colorbar(axes[2, 0].collections[0], ax=axes[2, 0])
cbar.set_label('Total Spent (£)', fontsize=10)

# Customer monetary value distribution
axes[2, 1].hist(customer_features['TotalSpent'], bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[2, 1].set_xlabel('Total Spent (£)', fontsize=12)
axes[2, 1].set_ylabel('Frequency', fontsize=12)
axes[2, 1].set_title('Customer Monetary Value Distribution', fontweight='bold')
axes[2, 1].set_xscale('log')
axes[2, 1].axvline(x=customer_features['TotalSpent'].median(), color='red', linestyle='--',
                  linewidth=2, label=f'Median: £{customer_features["TotalSpent"].median():,.0f}')
axes[2, 1].legend()
axes[2, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# 2. Summary statistics visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Data Summary Statistics', fontsize=14, y=1.02)

# Time-series statistics comparison
ts_stats = pd.DataFrame({
    'Metric': ['Mean', 'Median', 'Std', 'Min', 'Max'],
    'Daily Revenue': [
        daily_complete['DailyRevenue'].mean(),
        daily_complete['DailyRevenue'].median(),
        daily_complete['DailyRevenue'].std(),
        daily_complete['DailyRevenue'].min(),
        daily_complete['DailyRevenue'].max()
    ],
    'Monthly Revenue': [
        monthly_data['MonthlyRevenue'].mean(),
        monthly_data['MonthlyRevenue'].median(),
        monthly_data['MonthlyRevenue'].std(),
        monthly_data['MonthlyRevenue'].min(),
        monthly_data['MonthlyRevenue'].max()
    ]
})

x_pos = np.arange(len(ts_stats))
width = 0.35
axes[0].bar(x_pos - width/2, ts_stats['Daily Revenue'], width, label='Daily', alpha=0.7, color='steelblue', edgecolor='black')
axes[0].bar(x_pos + width/2, ts_stats['Monthly Revenue'], width, label='Monthly', alpha=0.7, color='coral', edgecolor='black')
axes[0].set_xlabel('Statistic', fontsize=12)
axes[0].set_ylabel('Value (£)', fontsize=12)
axes[0].set_title('Time-Series Statistics Comparison', fontweight='bold')
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(ts_stats['Metric'], rotation=45)
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].set_yscale('log')

# Customer feature statistics
customer_stats = pd.DataFrame({
    'Feature': ['Recency', 'Frequency', 'Total Spent'],
    'Mean': [
        customer_features['Recency'].mean(),
        customer_features['Frequency'].mean(),
        customer_features['TotalSpent'].mean()
    ],
    'Median': [
        customer_features['Recency'].median(),
        customer_features['Frequency'].median(),
        customer_features['TotalSpent'].median()
    ]
})

x_pos = np.arange(len(customer_stats))
axes[1].bar(x_pos - width/2, customer_stats['Mean'], width, label='Mean', alpha=0.7, color='teal', edgecolor='black')
axes[1].bar(x_pos + width/2, customer_stats['Median'], width, label='Median', alpha=0.7, color='purple', edgecolor='black')
axes[1].set_xlabel('Feature', fontsize=12)
axes[1].set_ylabel('Value', fontsize=12)
axes[1].set_title('Customer Feature Statistics', fontweight='bold')
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(customer_stats['Feature'], rotation=45)
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].set_yscale('log')

plt.tight_layout()
plt.show()

print("\n" + "=" * 80)
print("DATA VISUALIZATION COMPLETE")
print("=" * 80)


## Phase 3: ARIMA Time Series Modeling (Daily Revenue)

We now implement a **baseline ARIMA model** on the prepared **daily revenue** time series:
- Use `daily_complete` and the `DailyRevenue` column as the univariate series
- Check stationarity using the Augmented Dickey-Fuller (ADF) test
- Inspect ACF and PACF plots to guide (p, d, q) selection
- Split the data into train and test sets using a temporal split
- Fit an ARIMA model on the training set and forecast the test period
- Evaluate performance using MAE, RMSE, and MAPE



In [ ]:
# === ARIMA: prepare series and stationarity checks ===

# Build univariate daily revenue series
ts = daily_complete.set_index('Date')['DailyRevenue'].asfreq('D')

print("=" * 80)
print("ARIMA MODELING - DAILY REVENUE")
print("=" * 80)
print(f"Number of observations: {len(ts)}")
print(f"Start date: {ts.index.min()} | End date: {ts.index.max()}")

# Plot original series
fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(ts, color='steelblue')
ax.set_title("Daily Revenue Time Series", fontweight='bold')
ax.set_xlabel("Date")
ax.set_ylabel("Revenue (£)")
ax.grid(True, alpha=0.3)
plt.show()

# ADF helper
from statsmodels.tsa.stattools import adfuller

def adf_test(series, title=""):
    print(f"\nADF Test: {title}")
    result = adfuller(series.dropna(), autolag='AIC')
    labels = ['ADF Statistic', 'p-value', '# Lags Used', '# Observations Used']
    out = dict(zip(labels, result[0:4]))
    for k, v in out.items():
        print(f"  {k}: {v:.4f}")
    for key, value in result[4].items():
        print(f"  Critical Value {key}: {value:.4f}")
    if out['p-value'] < 0.05:
        print("  => Reject H0: Series is likely STATIONARY.")
    else:
        print("  => Fail to reject H0: Series is likely NON-STATIONARY.")

# Stationarity tests
adf_test(ts, title="DailyRevenue (Original)")

ts_diff = ts.diff().dropna()
adf_test(ts_diff, title="DailyRevenue (1st Difference)")

# ACF and PACF on differenced series
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
plot_acf(ts_diff, ax=axes[0], lags=30)
axes[0].set_title("ACF - Differenced Daily Revenue")
plot_pacf(ts_diff, ax=axes[1], lags=30, method='ywm')
axes[1].set_title("PACF - Differenced Daily Revenue")
plt.tight_layout()
plt.show()


In [ ]:
# === ARIMA: train/test split, fit and evaluation ===

# Temporal split (last 90 days as test)
test_size = 90
train_ts = ts.iloc[:-test_size]
test_ts = ts.iloc[-test_size:]

print("\nTrain/Test Split (ARIMA):")
print(f"  Train period: {train_ts.index.min()} to {train_ts.index.max()} ({len(train_ts)} days)")
print(f"  Test period : {test_ts.index.min()} to {test_ts.index.max()} ({len(test_ts)} days)")

order = (1, 1, 1)  # baseline choice; can be tuned
print(f"\nFitting ARIMA model with order={order} ...")
model = ARIMA(train_ts, order=order)
model_fit = model.fit()
print(model_fit.summary())

# Forecast over test period
start = len(train_ts)
end = len(train_ts) + len(test_ts) - 1
forecast = model_fit.predict(start=start, end=end, typ='levels')
forecast.index = test_ts.index

# Metrics
mae = mean_absolute_error(test_ts, forecast)
rmse = np.sqrt(mean_squared_error(test_ts, forecast))
mape = np.mean(np.abs((test_ts - forecast) / test_ts.replace(0, np.nan))) * 100

print("\n" + "=" * 80)
print("ARIMA MODEL PERFORMANCE (Daily Revenue)")
print("=" * 80)
print(f"MAE : {mae:,.2f}")
print(f"RMSE: {rmse:,.2f}")
print(f"MAPE: {mape:,.2f}%")
print("=" * 80)

# Plot actual vs forecast
plt.figure(figsize=(14, 4))
plt.plot(train_ts.index, train_ts, label='Train', color='gray', alpha=0.6)
plt.plot(test_ts.index, test_ts, label='Actual (Test)', color='steelblue')
plt.plot(forecast.index, forecast, label='Forecast (ARIMA)', color='coral', linestyle='--')
plt.title("ARIMA Daily Revenue Forecast - Train/Test", fontweight='bold')
plt.xlabel("Date")
plt.ylabel("Revenue (£)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Error distribution
errors = test_ts - forecast
plt.figure(figsize=(10, 4))
plt.hist(errors, bins=30, edgecolor='black', alpha=0.7)
plt.title("Forecast Error Distribution (ARIMA Test Set)", fontweight='bold')
plt.xlabel("Error (£)")
plt.ylabel("Frequency")
plt.grid(True, axis='y', alpha=0.3)
plt.show()


## Phase 3: Prophet Time Series Modeling (Daily Revenue)

We now implement a **Prophet** model on the daily revenue series to capture trend and seasonality and compare its performance with ARIMA.


In [ ]:
# === Prophet: model fitting, forecasting and evaluation ===

# Prepare data for Prophet: columns 'ds' (date) and 'y' (value)
prophet_df = daily_complete[['Date', 'DailyRevenue']].rename(columns={'Date': 'ds', 'DailyRevenue': 'y'})
prophet_df = prophet_df.sort_values('ds')

# Use same temporal split as ARIMA
prophet_train = prophet_df.iloc[:-test_size].copy()
prophet_test = prophet_df.iloc[-test_size:].copy()

print("=" * 80)
print("PROPHET MODELING - DAILY REVENUE")
print("=" * 80)
print(f"Train rows: {len(prophet_train)}, Test rows: {len(prophet_test)}")

m = Prophet(daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True)
m.fit(prophet_train)

future = prophet_test[['ds']].copy()
forecast_prophet = m.predict(future)

# Align predictions with test set
y_true = prophet_test['y'].values
y_pred = forecast_prophet['yhat'].values

mae_p = mean_absolute_error(y_true, y_pred)
rmse_p = np.sqrt(mean_squared_error(y_true, y_pred))
mape_p = np.mean(np.abs((y_true - y_pred) / np.where(y_true == 0, np.nan, y_true))) * 100

print("\n" + "=" * 80)
print("PROPHET MODEL PERFORMANCE (Daily Revenue)")
print("=" * 80)
print(f"MAE : {mae_p:,.2f}")
print(f"RMSE: {rmse_p:,.2f}")
print(f"MAPE: {mape_p:,.2f}%")
print("=" * 80)

# Plot actual vs Prophet forecast
plt.figure(figsize=(14, 4))
plt.plot(prophet_train['ds'], prophet_train['y'], label='Train', color='gray', alpha=0.6)
plt.plot(prophet_test['ds'], prophet_test['y'], label='Actual (Test)', color='steelblue')
plt.plot(forecast_prophet['ds'], forecast_prophet['yhat'], label='Forecast (Prophet)', color='green', linestyle='--')
plt.title("Prophet Daily Revenue Forecast - Train/Test", fontweight='bold')
plt.xlabel("Date")
plt.ylabel("Revenue (£)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Prophet components (trend/seasonality)
fig = m.plot_components(forecast_prophet)
plt.show()


## Phase 3: Customer Behavior Prediction (Churn / Activity)

We now build a **baseline customer behavior model**:
- Use the `customer_features` dataset (Recency, Frequency, Monetary, etc.)
- Define a simple churn/active label based on **Recency**
- Train a **logistic regression** classifier
- Evaluate using accuracy, precision, recall, F1, and ROC-AUC.


In [ ]:
# === Customer behavior model: Random Forest & XGBoost ===

from sklearn.ensemble import RandomForestClassifier

try:
    from xgboost import XGBClassifier  # type: ignore
    xgb_available = True
except Exception as e:  # pragma: no cover
    print("XGBoost not available in this environment. Skipping XGBoost model.")
    print("Error:", e)
    xgb_available = False

# Reuse the same supervised dataset (X, y) from the logistic regression step
X_rf = X
y_rf = y

X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(
    X_rf, y_rf, test_size=0.3, random_state=42, stratify=y_rf
)

print("\n" + "=" * 80)
print("CUSTOMER BEHAVIOR MODEL - RANDOM FOREST")
print("=" * 80)

rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    random_state=42,
    class_weight="balanced_subsample"
)
rf.fit(X_train_rf, y_train_rf)

rf_pred = rf.predict(X_test_rf)
rf_proba = rf.predict_proba(X_test_rf)[:, 1]

print("Random Forest Classification Report (Churn=1)")
print(classification_report(y_test_rf := y_test_rf if 'y_test_rf' in globals() else y_test, rf_pred, digits=3))

rf_roc_auc = roc_auc_score(y_test_rf, rf_proba)
print(f"Random Forest ROC-AUC: {rf_roc_auc:.3f}")

if xgb_available:
    print("\n" + "=" * 80)
    print("CUSTOMER BEHAVIOR MODEL - XGBOOST")
    print("=" * 80)

    xgb = XGBClassifier(
        n_estimators=300,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="binary:logistic",
        eval_metric="logloss",
        random_state=42
    )
    xgb.fit(X_train_rf, y_train_rf)

    xgb_pred = xgb.predict(X_test_rf)
    xgb_proba = xgb.predict_proba(X_test_rf)[:, 1]

    print("XGBoost Classification Report (Churn=1)")
    print(classification_report(y_test_rf, xgb_pred, digits=3))

    xgb_roc_auc = roc_auc_score(y_test_rf, xgb_proba)
    print(f"XGBoost ROC-AUC: {xgb_roc_auc:.3f}")

print("\nCompare the performance of Logistic Regression, Random Forest, and XGBoost to choose the best model for deployment.")



In [ ]:
# === Customer behavior model: churn vs active ===

# Label definition: customers with recent purchase are "active" (0), others "churned" (1)
# Here we mark churned if Recency > 90 days (tunable threshold)

customer_model_df = customer_features.copy().dropna(subset=['Recency', 'Frequency', 'TotalSpent'])
customer_model_df['Churned'] = (customer_model_df['Recency'] > 90).astype(int)

X = customer_model_df[['Recency', 'Frequency', 'TotalSpent', 'AvgDaysBetweenPurchases']].fillna(0)
y = customer_model_df['Churned']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print("=" * 80)
print("CUSTOMER BEHAVIOR MODEL - LOGISTIC REGRESSION")
print("=" * 80)
print(f"Train size: {len(X_train)}, Test size: {len(X_test)}")
print(f"Churn rate (overall): {y.mean():.3f}")

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_scaled, y_train)

y_pred = log_reg.predict(X_test_scaled)
y_proba = log_reg.predict_proba(X_test_scaled)[:, 1]

print("\nClassification Report (Churn=1)")
print(classification_report(y_test, y_pred, digits=3))

roc_auc = roc_auc_score(y_test, y_proba)
print(f"ROC-AUC: {roc_auc:.3f}")

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

fig, ax = plt.subplots(figsize=(5, 4))
im = ax.imshow(cm, cmap='Blues')
ax.set_title('Confusion Matrix - Customer Churn Model', fontweight='bold')
ax.set_xlabel('Predicted label')
ax.set_ylabel('True label')

for (i, j), v in np.ndenumerate(cm):
    ax.text(j, i, str(v), ha='center', va='center', color='black', fontweight='bold')

ax.set_xticks([0, 1])
ax.set_yticks([0, 1])
ax.set_xticklabels(['Active (0)', 'Churned (1)'])
ax.set_yticklabels(['Active (0)', 'Churned (1)'])
plt.colorbar(im, ax=ax)
plt.tight_layout()
plt.show()


## Phase 3: Model Selection & Hyperparameter Tuning (Customer Behavior)

To maximise predictive performance while avoiding overfitting, we compare and tune **Random Forest** and **XGBoost** on the churn label using **ROC-AUC** with stratified cross‑validation. The best‑performing model can then be promoted for deployment in the dashboard/backend.


In [ ]:
# === Model selection & hyperparameter tuning (ROC-AUC, CV) ===

print("=" * 80)
print("MODEL SELECTION & HYPERPARAMETER TUNING - CUSTOMER BEHAVIOR")
print("=" * 80)

# Use the full supervised dataset X, y defined above (Recency, Frequency, Monetary, etc.)
X_tune = X.copy()
y_tune = y.copy()

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Random Forest search space
rf_base = RandomForestClassifier(random_state=42, class_weight="balanced_subsample")
rf_param_dist = {
    "n_estimators": randint(150, 400),
    "max_depth": randint(3, 12),
    "min_samples_split": randint(2, 10),
    "min_samples_leaf": randint(1, 6),
    "max_features": ["sqrt", "log2", 0.5, 0.8]
}

rf_search = RandomizedSearchCV(
    rf_base,
    param_distributions=rf_param_dist,
    n_iter=20,
    scoring="roc_auc",
    cv=cv,
    n_jobs=-1,
    random_state=42,
    verbose=1
)

rf_search.fit(X_tune, y_tune)
print("\nBest Random Forest params:", rf_search.best_params_)
print(f"Best CV ROC-AUC (RF): {rf_search.best_score_:.3f}")

best_rf = rf_search.best_estimator_

if xgb_available:
    # XGBoost search space
    xgb_base = XGBClassifier(
        objective="binary:logistic",
        eval_metric="logloss",
        random_state=42,
        tree_method="hist"
    )

    xgb_param_dist = {
        "n_estimators": randint(200, 500),
        "max_depth": randint(3, 8),
        "learning_rate": uniform(0.01, 0.2),
        "subsample": uniform(0.6, 0.4),
        "colsample_bytree": uniform(0.6, 0.4)
    }

    xgb_search = RandomizedSearchCV(
        xgb_base,
        param_distributions=xgb_param_dist,
        n_iter=20,
        scoring="roc_auc",
        cv=cv,
        n_jobs=-1,
        random_state=42,
        verbose=1
    )

    xgb_search.fit(X_tune, y_tune)
    print("\nBest XGBoost params:", xgb_search.best_params_)
    print(f"Best CV ROC-AUC (XGB): {xgb_search.best_score_:.3f}")

    best_xgb = xgb_search.best_estimator_

print("\nSelect the model (RF or XGB) with the higher CV ROC-AUC as the final production model.")

